In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 데이터 불러오기
df = pd.read_csv("C:\\Users\\user\\Desktop\\data2.csv")

# 각 컬럼을 리스트로 변환
a1 = df['과목'].tolist()  # '과목' 컬럼을 리스트로 변환
a2 = df['과목2'].tolist()  # '과목2' 컬럼을 리스트로 변환
a3 = df['과목3'].tolist()  # '과목3' 컬럼을 리스트로 변환

# 입력 데이터 (a1과 a2를 사용하여 2차원 데이터로 변환)
input_data = [[a1[i], a2[i]] for i in range(len(a1))]
data = torch.tensor(input_data, dtype=torch.float32)

# 출력 데이터 (a3 값에 따라 0 또는 1로 분류, 예시로 짝수는 0, 홀수는 1)
target = torch.tensor([[0] if a3[i] % 2 == 0 else [1] for i in range(len(a3))], dtype=torch.float32)

# XOR 모델 정의
class XORModel(nn.Module):
    def __init__(self):
        super(XORModel, self).__init__()
        self.fc1 = nn.Linear(2, 100)  # 입력 차원 2, 출력 차원 100
        self.fc2 = nn.Linear(100, 1)  # 출력 차원 1

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # 첫 번째 층에 ReLU 활성화 함수 적용
        x = self.fc2(x)  # 두 번째 층 (출력)
        return x

# 모델, 손실 함수, 옵티마이저 정의
model = XORModel()
criterion = nn.BCEWithLogitsLoss()  # 이진 크로스 엔트로피 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
epochs = 100000  # 학습 에폭 수
for epoch in range(epochs):
    model.train()
    
    # 순전파
    outputs = model(data).squeeze()  # 예측값
    loss = criterion(outputs, target.squeeze())  # 손실 계산

    # 역전파
    optimizer.zero_grad()  # 기울기 초기화
    loss.backward()  # 기울기 계산
    optimizer.step()  # 가중치 업데이트
    
    if (epoch + 1) % 10000 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    predictions = model(data).squeeze()  # 예측값
    predicted_classes = (predictions > 0.5).float()  # 0.5를 기준으로 0 또는 1로 변환

    # 정확도 계산
    correct = (predicted_classes == target.squeeze()).sum().item()
    total = target.size(0)
    accuracy = (correct / total) * 100
    
    # 결과 출력
    print(f"Predictions: {predicted_classes.tolist()}")
    print(f"Target: {target.tolist()}")
    
    # 맞춘 횟수 계산
    correct_predictions = (predicted_classes == target.squeeze()).sum().item()

    # 출력 형식 맞추기
    print(f"[{predicted_classes.tolist()}, {correct_predictions}] 1의 개수 맞춘 횟수 : {correct_predictions}")

    # 정확도 시각화
    print(f"Accuracy: {accuracy:.2f}%")

    # 그래프 그리기
    plt.figure(figsize=(12, 6))

    # 실제 값과 예측 값의 비교
    plt.subplot(1, 2, 1)
    plt.scatter(range(len(target)), target.squeeze(), color='blue', label='Actual', alpha=0.6)
    plt.scatter(range(len(predicted_classes)), predicted_classes.squeeze(), color='red', label='Predicted', alpha=0.6)

    plt.title('Actual vs Predicted')
    plt.xlabel('Sample Index')
    plt.ylabel('Output Value (0 or 1)')
    plt.legend(loc='best')

    # 혼동 행렬 (Confusion Matrix)
    cm = confusion_matrix(target.squeeze().numpy(), predicted_classes.squeeze().numpy())
    plt.subplot(1, 2, 2)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Pred: 0', 'Pred: 1'], yticklabels=['True: 0', 'True: 1'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')

    # 그래프를 한 화면에 출력
    plt.tight_layout()
    plt.show()


ModuleNotFoundError: No module named 'sklearn'